# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Jorge Luis Olarte Vargas  <br>
Url: https://github.com/jolarte8811/Algoritmos_de_Optimizaci-n.git <br>
Problema:
> 1. Sesiones de doblaje <br>



Descripción del problema:Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son

#### Número de actores: 10
#### Número de tomas : 30
#### Actores/Tomas : https://bit.ly/36D8IuK
....

(*) La respuesta es obligatoria







In [52]:
import math
import pandas as pd
import numpy as np
import os
import random
from itertools import permutations

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>




¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.





Respuesta

La cantidad de posibilidades sin tener en cuenta las restriciones es: 2**30


In [53]:
print(f"La cantidad de posibilidades sin tener en cuenta las restriciones es {2**30}")

La cantidad de posibilidades sin tener en cuenta las restriciones es 1073741824


In [54]:
print(f"las posibilidades teniendo en cuenta las restricciones son 30!/(24!*6!)")
math.factorial(30)/(math.factorial(24)*math.factorial(6))

las posibilidades teniendo en cuenta las restricciones son 30!/(24!*6!)


593775.0

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Para este problema elegí, una estuctura de lista [np.array] con el rango 1 a 30, la cual representa las tomas posibles, y unicamente emplear la matriz (30, 10), en la evaluación de coste, debido:

1. Legibilidad: para mi es mas claro trabajar con una respuesta del tipo, para el dia 1 seran las tomas [1,3,5,7,9,11], que tener una matriz de unos y ceros.

2. Eficiencia: la estructura de datos np.array esta optimizada a partir de un lenguaje de bajo nivel, y el Broadcasting de numpy, permite que operar sobre una lista tenga menor coste computacional.

3. Flexibilidad: existen operaciones en numpy que realizarlas en una lista tradicional son menos directas, por ejemplo contar los elementos no ceros de una lista: np.count_nonzero(s), la cual es empleada en la función de coste

Si inicié con una estructura matricial en Python clásico, pero me **arrepentí.**

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?



(*)¿Es un problema de maximización o minimización?




Respuesta

función objetivo
sumatoria de coste de cada dia programado, donde el coste de un dia es equivalente al numero de actores que intervienen ese dia. siendo un problema de  minimización de coste, que tambien puede ser planteado como minimizar la cantidad de actores que intervienen cada dia,

**Este es un problema de minización de coste**

In [55]:
def costo(df,programacion,nueva_toma=0):
  s = np.zeros(10)
  if nueva_toma:
    programacion2=np.append(programacion,nueva_toma)
  for i in programacion:
    s=s+df.loc[i]
  cost = np.count_nonzero(s)
  return cost

def costo_tomas(df,tomas):
  c = 0
  for i in (tomas):
    c = c + costo(df,i)
  return(c)

def solucion_Fuerza_B_multiple(df,soluciones):
  mejor_costo = 100
  mejor_solucion = []

  for i in range(len(soluciones)):

    costo1 = costo_tomas(df,soluciones[i])
    #if costo1 <38:
      #print(costo1,i)
      #print(mejor_solucion)
    if costo1<mejor_costo:
      mejor_costo = costo1
      mejor_solucion = soluciones[i]

  return(mejor_solucion)

def formato_respuesta(solucion1):
  solucion= pd.DataFrame(solucion1)
  index=["dia1","dia2","dia3","dia4","dia5"]
  columns=["T1","T2","T3","T4","T5","T6"]
  solucion.index=index
  solucion.columns=columns
  return(solucion)


Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

## Importar Datos

In [56]:
path = os.path.join("/content/Datos problema doblaje(30 tomas, 10 actores).xlsx")

try:
  df = pd.read_excel(path)
  df.columns = ["Toma","Act01","Act02","Act03","Act04","Act05","Act06","Act07","Act08","Act09","Act10","Act11","Total" ]
  df.drop("Act11",axis=1,inplace=True)
  df.drop([31,32],axis=0,inplace=True)
  df.drop(0,axis=0,inplace=True)
  df.index=df["Toma"]
  df.drop("Toma",axis=1,inplace=True)
  df.drop("Total",axis=1,inplace=True)
  df
except FileNotFoundError:
  print("---- Archivo no encontrado por favor cargarlo----")

## Solucion fuerza bruta (ordenada)
La solución propuesta es grabar las tomas en orden y quedarse con la mejor

In [57]:
def probar_todas_las_matrices(filas, columnas, max_intentos=100):
    # Generamos todas las permutaciones de los números del 1 al 30
    numeros = list(range(1, 31))
    todas_las_combinaciones = permutations(numeros)


    matrices_validas = []
    intentos = 0

    # Verificamos cada combinación y almacenamos las matrices válidas hasta alcanzar el límite de intentos
    for combinacion in todas_las_combinaciones:
        matriz = []
        for i in range(filas):
            fila = list(combinacion[i*columnas : (i+1)*columnas])
            matriz.append(fila)

        # Verificamos si la matriz es válida (sin números repetidos)
        numeros_en_matriz = [num for fila in matriz for num in fila]
        if len(numeros_en_matriz) == len(set(numeros_en_matriz)):
            matrices_validas.append(matriz)

        # Detenemos el proceso después de alcanzar el límite de intentos
        intentos += 1
        if intentos >= max_intentos:
            break

    return matrices_validas

n_iteraciones = 50000
Soluciones_Fuerza_Bruta = probar_todas_las_matrices(5, 6, max_intentos=n_iteraciones)
costo_primera_sol_Fuerza_bruta = costo_tomas(df,Soluciones_Fuerza_Bruta[0])
Mejor_solucion_Fuerzabrura=solucion_Fuerza_B_multiple(df,Soluciones_Fuerza_Bruta)
costo_Mejor_sol_Fuerza_bruta = costo_tomas(df,Mejor_solucion_Fuerzabrura)

## Resultado
print("La primera solución propuesta por fuerza bruta es")
display(formato_respuesta(Soluciones_Fuerza_Bruta[0]))
print(f"Con una costo de {costo_primera_sol_Fuerza_bruta }")
print("-------//----------//------")
print(f"Luego de {n_iteraciones} iteraciones de fuerza bruta la mejor solución es")
display(formato_respuesta(Mejor_solucion_Fuerzabrura))
print(f"Con un costo de {costo_Mejor_sol_Fuerza_bruta}")




La primera solución propuesta por fuerza bruta es


,T1,T2,T3,T4,T5,T6
dia1,1,2,3,4,5,6
dia2,7,8,9,10,11,12
dia3,13,14,15,16,17,18
dia4,19,20,21,22,23,24
dia5,25,26,27,28,29,30


Con una costo de 38
-------//----------//------
Luego de 50000 iteraciones de fuerza bruta la mejor solución es


,T1,T2,T3,T4,T5,T6
dia1,1,2,3,4,5,6
dia2,7,8,9,10,11,12
dia3,13,14,15,16,17,18
dia4,19,20,21,23,24,29
dia5,22,25,26,27,28,30


Con un costo de 36


## Solucion fuerza bruta (Aleatoria)
La solución propuesta es grabar las tomas en orden aleatorio y quedarse con la mejor

In [58]:
def solucion_aleatorea(tomas=30,maximo_tomas=6):
  lista=list(range(tomas))

  programacion=[]
  dia=np.array([])
  selecionados=[]
  for i in range(tomas):
    l = random.choice(list(set(lista)-set(selecionados)))
    selecionados.append(l)
    dia=np.append(dia,l+1)
    if (i+1)%maximo_tomas==0:
      programacion.append(dia)
      dia=[]

  return(programacion)

def solucion_aleatorea_multiple(df,intentos=100):
  mejor_costo = 100
  mejor_solucion = []

  for i in range(intentos):
    candidato = solucion_aleatorea()
    costo1 = costo_tomas(df,candidato)
    if costo1<mejor_costo:
      mejor_costo = costo1
      mejor_solucion = candidato
  return(mejor_solucion)

Mejor_solucion_alea=solucion_aleatorea_multiple(df,50000)
costo_aleatorio = costo_tomas(df,Mejor_solucion_alea)

## Resultado
print(f"Luego de {n_iteraciones} iteraciones de fuerza bruta aleatoria la mejor solución es")
display(formato_respuesta(Mejor_solucion_alea))
print(f"Con un costo de {costo_aleatorio}")


Luego de 50000 iteraciones de fuerza bruta aleatoria la mejor solución es


,T1,T2,T3,T4,T5,T6
dia1,20.0,21.0,23.0,1.0,14.0,9.0
dia2,27.0,6.0,13.0,16.0,7.0,25.0
dia3,4.0,28.0,11.0,3.0,15.0,5.0
dia4,19.0,30.0,2.0,17.0,10.0,26.0
dia5,18.0,24.0,12.0,29.0,22.0,8.0


Con un costo de 32


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Tanto la solucion **solucion_Fuerza_B_multiple** como la **solucion_aleatorea_multiple** tienen complejidad O(n!)






(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

# Solucion greedy
La propuesta es grabar las tomas agregando desde la toma que menor costo impacte y mayor número de actores tenga  

In [59]:
def mejor_siguiente_toma(programacion,tomas_pendientes,df=df):
  if programacion.shape[0]==0:
    programacion=np.append(programacion,tomas_pendientes[0])
    tomas_pendientes=np.delete(tomas_pendientes,0)
    return(programacion,tomas_pendientes)
  else:
    mejor_costo=11
    posicion=-1
    for i in tomas_pendientes:
      posicion = posicion+1
      programacion2 = np.append(programacion,i)
      c=costo(df,programacion2)

      if c<mejor_costo:
        mejor_costo=c
        mejor_posicion=posicion
    programacion2=np.append(programacion,tomas_pendientes[mejor_posicion])
    tomas_pendientes=np.delete(tomas_pendientes,mejor_posicion)
    return(programacion2,tomas_pendientes)


def programa_diario(t,df,maximo_tomas_dia=6):
  prog = np.array([])
  for i in range(maximo_tomas_dia):
    prog,t = mejor_siguiente_toma(prog,t,df)
  return(prog,t)

def programa_general(df,t,dias=5):
  programa=[]
  c = 0
  for i in range(dias):
    dia,t=programa_diario(t,df)
    programa.append(dia)
    c = c +costo(df,dia)

  return(programa,c)

tomas_orden=df.sum(axis=1).sort_values(ascending=False) # Ordenamos las tomas con mayor numero de actores Primero
tomas_orden = tomas_orden.reset_index()["Toma"].values

tomas = tomas_orden.copy()

sol_greedy=programa_general(df,tomas)

print(f"La solución mediante algoritmo Greddy es")
display(formato_respuesta(sol_greedy[0]))


print(f"Con un costo total de {sol_greedy[1]}")

La solución mediante algoritmo Greddy es


,T1,T2,T3,T4,T5,T6
dia1,1.0,22.0,20.0,7.0,6.0,9.0
dia2,11.0,23.0,19.0,17.0,4.0,3.0
dia3,12.0,8.0,14.0,24.0,28.0,18.0
dia4,10.0,15.0,29.0,26.0,13.0,2.0
dia5,25.0,16.0,30.0,5.0,27.0,21.0


Con un costo total de 31


In [60]:
tomas

array([ 1, 11, 12,  4, 10, 22, 26, 20, 25,  7,  6,  9,  3, 13,  5, 15,  2,
       29,  8, 14, 24, 28, 27, 16, 23, 21, 19, 18, 17, 30])

(*)Calcula la complejidad del algoritmo

**Respuesta** El algoritmo **Greedy**  tiene una complejidad lineal O(n**2)

**Justificación**

El algoritmo greedy diseñado, supera los resultados empleados por el algoritmo de fuerza bruta con 50.000 iteraciones y el algoritmo de fuerza bruta **aleatoria** con 50.000 iteraciones, y tiene una menor complejidad, a continuación un resumen de  los resultados:

In [61]:
print(f"el algoritmo Fuerza bruta con 50.000 iteraciones, logra un costo de  {costo_Mejor_sol_Fuerza_bruta} ")
print(f"el algoritmo Fuerza bruta aleatoria con 50.000 iteraciones, logra un costo de  {costo_aleatorio}")
print(f"el algoritmo Greddy con 1 iteracion, logra un costo de  {sol_greedy[1]}")

el algoritmo Fuerza bruta con 50.000 iteraciones, logra un costo de  36 
el algoritmo Fuerza bruta aleatoria con 50.000 iteraciones, logra un costo de  32
el algoritmo Greddy con 1 iteracion, logra un costo de  31


Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

**Respuesta Juego de Datos**

In [62]:
indice=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
col = ["Act01","Act02","Act03","Act04","Act05","Act06","Act07","Act08","Act09","Act10"]
juego_dados = pd.DataFrame(np.random.randint(2,size=(30,10)),index=indice)
juego_dados.columns=col
juego_dados.to_excel("Juego_Dados.xlsx")
juego_dados

,Act01,Act02,Act03,Act04,Act05,Act06,Act07,Act08,Act09,Act10
1,1,0,1,0,1,1,0,1,1,1
2,0,0,1,1,0,0,0,0,1,0
3,0,1,0,0,0,1,1,0,1,0
4,0,0,0,1,0,1,1,1,1,0
5,0,0,1,0,0,0,1,1,1,0
6,0,0,1,1,1,0,0,0,0,1
7,0,1,0,1,0,0,0,1,1,0
8,0,1,1,0,0,1,1,1,0,1
9,0,0,1,0,0,0,1,0,1,1
10,0,1,0,1,1,0,0,0,1,1


Aplica el algoritmo al juego de datos generado

Respuesta

In [63]:
tomas_orden2=juego_dados.sum(axis=1).sort_values(ascending=False) # Ordenamos las tomas con mayor numero de actores Primero
tomas_orden2 = tomas_orden2.index
tomas2 = tomas_orden2.copy()
sol_greedy2=programa_general(juego_dados,tomas2)
display(formato_respuesta(sol_greedy2[0]))
print(f"Solución Greedy Juego de datos Con un costo total de {sol_greedy2[1]}")



,T1,T2,T3,T4,T5,T6
dia1,26.0,14.0,29.0,8.0,17.0,22.0
dia2,1.0,28.0,11.0,21.0,15.0,23.0
dia3,18.0,5.0,25.0,7.0,2.0,27.0
dia4,13.0,6.0,19.0,16.0,10.0,12.0
dia5,30.0,24.0,4.0,3.0,20.0,9.0


Solución Greedy Juego de datos Con un costo total de 42


In [64]:
Mejor_solucion_alea2=solucion_aleatorea_multiple(juego_dados,50000)
costo_aleatorio2 = costo_tomas(juego_dados,Mejor_solucion_alea)
## Resultado
print(f"Luego de {n_iteraciones} iteraciones de fuerza bruta aleatoria la mejor solución es")
display(formato_respuesta(Mejor_solucion_alea2))
print(f"Con un costo de {costo_aleatorio2}")

Luego de 50000 iteraciones de fuerza bruta aleatoria la mejor solución es


,T1,T2,T3,T4,T5,T6
dia1,5.0,24.0,20.0,30.0,18.0,16.0
dia2,26.0,9.0,23.0,14.0,8.0,15.0
dia3,13.0,27.0,25.0,19.0,11.0,29.0
dia4,3.0,22.0,21.0,4.0,10.0,17.0
dia5,28.0,12.0,6.0,1.0,7.0,2.0


Con un costo de 49


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

**Respuesta**

 Alejandro Suárez Lamadrid y Antonio Suárez Jiménez, Bucles eficientes con Itertools,  
  https://python-para-impacientes.blogspot.com/2015/08/bucles-eficientes-con-itertools.html

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

**Respuesta**

El paso a seguir es buscar en los vecindarios cercanos de la solución Greddy, y aplicar GRASP, lo que probablemente genere un mejor resultado aumentando la complejidad o probar con algunos dias de menos de 6 tomas.

La solución generada por el algoritmo Greedy es suficientemente buena, a un coste computacional bajo, y como vimos en la prueba de Juego de dados es flexible para otras poblaciones y condiciones.